In [31]:
from transformers import pipeline, AutoModel, AutoTokenizer, BertTokenizer, BertForQuestionAnswering
import transformers
import torch
import pandas as pd

# Basic QA on covid-19 wikipedia 

In [39]:
DATA_DIRECTORY = "/home/xcs224u_student/notebooks/cs224u/github_repo/data"
nlp_qa = pipeline('question-answering')

In [3]:
nlp_qa(context='Hugging Face is a French company based in New-York.', question='Where is based Hugging Face ?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3581.81it/s]


{'score': 0.9632967461141817, 'start': 42, 'end': 50, 'answer': 'New-York.'}

In [4]:
CONVID_ARTICLE="Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).[8] The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.[9][10] Common symptoms include fever, cough and shortness of breath.[5] Other symptoms may include fatigue, muscle pain, diarrhoea, sore throat, loss of smell and abdominal pain.[5][11][12] The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days.[5][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.[9][14] As of 10 April 2020, more than 1.67 million[7] cases have been reported in more than 200 countries and territories,[15] resulting in more than 101,000 deaths.[7] More than 372,000 people have recovered. The virus is mainly spread between people during close contact,[a] often via small droplets produced during coughing,[b] sneezing, or talking.[6][16][18] While these droplets are produced when breathing out, they usually fall to the ground or surfaces rather than being infectious over large distances.[6][19][20] People may also become infected by touching a contaminated surface and then their face.[6][16] The virus can survive on surfaces for up to 72 hours.[21] Coronavirus is most contagious during the first three days after onset of symptoms, although spread may be possible before symptoms appear and in later stages of the disease.The standard method of diagnosis is by real-time reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab.[23] The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.Recommended measures to prevent infection include frequent hand washing, maintaining physical distance from others (especially from those with symptoms), covering coughs and sneezes with a tissue or inner elbow and keeping unwashed hands away from the face.[26][27] The use of masks is recommended for those who suspect they have the virus and their caregivers.[28] Recommendations for mask use by the general public vary, with some authorities recommending against their use, some recommending their use and others requiring their use.[29][30][31] Currently, there is no vaccine or specific antiviral treatment for COVID-19.[6] Management involves treatment of symptoms, supportive care, isolation and experimental measures.The World Health Organization (WHO) declared the 2019–20 coronavirus outbreak a Public Health Emergency of International Concern (PHEIC)[33][34] on 30 January 2020 and a pandemic on 11 March 2020.[10] Local transmission of the disease has been recorded in many countries across all six WHO regions.[35]"

In [5]:
nlp_qa(context=CONVID_ARTICLE, question='Where is based Hugging Face ?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9000.65it/s]


{'score': 0.30009528747927305,
 'start': 189,
 'end': 234,
 'answer': "Wuhan, the capital of China's Hubei province,"}

In [6]:
nlp_qa(context=CONVID_ARTICLE, question='What is covid-19 ?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5518.82it/s]


{'score': 0.25412974718281767,
 'start': 39,
 'end': 104,
 'answer': 'an infectious disease caused by severe acute respiratory syndrome'}

In [7]:
nlp_qa(context=CONVID_ARTICLE, question='What are the symptoms')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8981.38it/s]


{'score': 0.23947629162223372,
 'start': 352,
 'end': 392,
 'answer': 'fever, cough and shortness of breath.[5]'}

# get baseline model working squad

In [8]:
# we can pass file paths to from_pretrained to load pretained weights, not sure what dataset they used to train these, I think SQUAD V1
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [9]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_ids = tokenizer.encode(question, text) # add deimiter tokens, return vocab index

token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))] #

#Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]: 0 corresponds to a sentence A token,
#1 corresponds to a sentence B token

start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(input_ids)
print(token_type_ids)
print(start_scores)
print(end_scores)
print(all_tokens)
answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

assert answer == "a nice puppet"

[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 27227, 2001, 1037, 3835, 13997, 102]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
tensor([[-4.0490, -4.1443, -5.7222, -2.4875, -5.2655, -8.2328, -4.0487,  0.3495,
         -3.8018, -1.2565,  7.1202,  5.9041,  3.4469, -4.0490]],
       grad_fn=<SqueezeBackward1>)
tensor([[ 1.1705, -4.7681, -5.0704, -4.3949, -1.5484, -4.6083,  1.1709, -3.3678,
         -0.8850, -3.4272, -0.7656,  2.3183,  7.1591,  1.1703]],
       grad_fn=<SqueezeBackward1>)
['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'nice', 'puppet', '[SEP]']


# baseline model  SQUAD using pipelines

In [11]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer


# Question answering pipeline, specifying the checkpoint identifier
qa_squad_pipeline = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')


In [15]:
qa_squad_pipeline(context='i love u, Hugging Face is a French company based in New-York.', question='Where is based Hugging Face ?')



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5907.47it/s]


{'score': 0.9680584762547824, 'start': 52, 'end': 60, 'answer': 'New-York.'}

d

# evaluate the model on squad 2.0: Generate evaluation JSON from dev set and and feed it to evaluate-v2.0.py to obtain metrics

In [91]:
from transformers.data.processors import squad
processor = squad.SquadV2Processor()
dev_squad_2 = processor.get_dev_examples(DATA_DIRECTORY, "squad/dev-v2.0.json")
#squad datasets: https://rajpurkar.github.io/SQuAD-explorer/

100%|██████████| 35/35 [00:05<00:00,  6.14it/s]


In [57]:
import json
def save_evaluation_squad_json(squadExamples, predictions, filename=DATA_DIRECTORY+'/evaluation.json'):
    obj = {};
    index =0
    for example in squadExamples:
        obj[example.qas_id] = predictions[index]["answer"]
        index += 1;
    with open(filename, 'w') as fout:
        json.dump(obj, fout)

In [58]:
import os;
SQUAD2_BASIC_EVALUATION_FILENAME = "/squad/evaluation_dev_basic_squad1.json"
if os.path.isfile(DATA_DIRECTORY+SQUAD2_BASIC_EVALUATION_FILENAME) == False:
    print("evaluation file doesnt exist, creating it")
    qa_squad_predictions = qa_squad_pipeline(dev_squad_2)
    save_evaluation_squad_json(dev_squad_2, qa_squad_predictions, DATA_DIRECTORY+SQUAD2_BASIC_EVALUATION_FILENAME)
    

evaluation file doesnt exist, creating it


# metrics obtained squad 2

In [82]:
import os
import subprocess
SCRIPT_PATH = DATA_DIRECTORY+"/evaluate-v2.0.py"
DEV_V2_PATH = DATA_DIRECTORY + "/squad/dev-v2.0.json"
EVALUATION_DEV_BASIC_PATH = DATA_DIRECTORY + SQUAD2_BASIC_EVALUATION_FILENAME
print(os.system("python " + SCRIPT_PATH + " " + DEV_V2_PATH + " " + EVALUATION_DEV_BASIC_PATH))
# fixme: need to run this on shell manually and copy results, cause this doesn't get console output

0


In [88]:
#python evaluate-v2.0.py squad/dev-v2.0.json squad/evaluation_dev_basic.json

{
  "exact": 39.15606839046576,
  "f1": 43.14232380992385,
  "total": 11873,
  "HasAns_exact": 78.42442645074225,
  "HasAns_f1": 86.40836885884374,
  "HasAns_total": 5928,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 5945
}

{'exact': 39.15606839046576,
 'f1': 43.14232380992385,
 'total': 11873,
 'HasAns_exact': 78.42442645074225,
 'HasAns_f1': 86.40836885884374,
 'HasAns_total': 5928,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 5945}

BERT was finetuned on SQUAD V1 so it fails to predict no-answers, grab some examples to ilustrate this. how can I get it to predict no-answers? need it for baseline, checkout this paper: https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15759479.pdf

# Evaluate on squad 1

In [92]:
#import os;
#dev_squad_2 = processor.get_dev_examples(DATA_DIRECTORY, "squad/dev-v1.0.json")

#SQUAD1_BASIC_EVALUATION_FILENAME = "/squad/evaluation_dev_basic_squad1.json"
#if os.path.isfile(DATA_DIRECTORY+SQUAD1_BASIC_EVALUATION_FILENAME) == False:
 #   print("evaluation file doesnt exist, creating it")
  #  qa_squad1_predictions = qa_squad_pipeline(dev_squad_1)
   # save_evaluation_squad_json(dev_squad_1, qa_squad1_predictions, DATA_DIRECTORY+SQUAD1_BASIC_EVALUATION_FILENAME)
    

# Baseline model using BioBERT